# Measure Pitch, HNR, Jitter, Shimmer, Formants, and Estimate VTL

## Import the external modules

In [21]:
#!/usr/bin/env python3
import glob
import numpy as np
import pandas as pd
import parselmouth 
import statistics


from parselmouth.praat import call
from scipy.stats.mstats import zscore
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [22]:
path_to_wav_file = r"C:\Users\user\Desktop\MA Sprache und Kommunikation\2_Semester\Sprechstile\Angehelescu\Angehelescu*.wav"

## This function measures duration, pitch, HNR, jitter, and shimmer

In [23]:
# This is the function to measure source acoustics using default male parameters.

def measurePitch(voiceID, f0min, f0max, unit):
    sound = parselmouth.Sound(voiceID) # read the sound
    duration = call(sound, "Get total duration") # duration
    pitch = call(sound, "To Pitch", 0.0, f0min, f0max) #create a praat pitch object
    meanF0 = call(pitch, "Get mean", 0, 0, unit) # get mean pitch
    stdevF0 = call(pitch, "Get standard deviation", 0 ,0, unit) # get standard deviation
    harmonicity = call(sound, "To Harmonicity (cc)", 0.01, f0min, 0.1, 1.0)
    hnr = call(harmonicity, "Get mean", 0, 0)
    pointProcess = call(sound, "To PointProcess (periodic, cc)", f0min, f0max)
    localJitter = call(pointProcess, "Get jitter (local)", 0, 0, 0.0001, 0.02, 1.3)
    localabsoluteJitter = call(pointProcess, "Get jitter (local, absolute)", 0, 0, 0.0001, 0.02, 1.3)
    rapJitter = call(pointProcess, "Get jitter (rap)", 0, 0, 0.0001, 0.02, 1.3)
    ppq5Jitter = call(pointProcess, "Get jitter (ppq5)", 0, 0, 0.0001, 0.02, 1.3)
    ddpJitter = call(pointProcess, "Get jitter (ddp)", 0, 0, 0.0001, 0.02, 1.3)
    localShimmer =  call([sound, pointProcess], "Get shimmer (local)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    localdbShimmer = call([sound, pointProcess], "Get shimmer (local_dB)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    apq3Shimmer = call([sound, pointProcess], "Get shimmer (apq3)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    aqpq5Shimmer = call([sound, pointProcess], "Get shimmer (apq5)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    apq11Shimmer =  call([sound, pointProcess], "Get shimmer (apq11)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    ddaShimmer = call([sound, pointProcess], "Get shimmer (dda)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    
    return duration, meanF0, stdevF0, hnr, localJitter, localabsoluteJitter, rapJitter, ppq5Jitter, ddpJitter, localShimmer, localdbShimmer, apq3Shimmer, aqpq5Shimmer, apq11Shimmer, ddaShimmer

## This function measures formants at each glottal pulse

Puts, D. A., Apicella, C. L., & Cárdenas, R. A. (2012). Masculine voices signal men's threat potential in forager and industrial societies. Proceedings of the Royal Society of London B: Biological Sciences, 279(1728), 601-609.

Adapted from: DOI 10.17605/OSF.IO/K2BHS

In [24]:
# This function measures formants using Formant Position formula
def measureFormants(sound, wave_file, f0min,f0max):
    sound = parselmouth.Sound(sound) # read the sound
    pitch = call(sound, "To Pitch (cc)", 0, f0min, 15, 'no', 0.03, 0.45, 0.01, 0.35, 0.14, f0max)
    pointProcess = call(sound, "To PointProcess (periodic, cc)", f0min, f0max)
    
    formants = call(sound, "To Formant (burg)", 0.0025, 5, 5000, 0.025, 50)
    numPoints = call(pointProcess, "Get number of points")

    f1_list = []
    f2_list = []
    f3_list = []
    f4_list = []
    
    # Measure formants only at glottal pulses
    for point in range(0, numPoints):
        point += 1
        t = call(pointProcess, "Get time from index", point)
        f1 = call(formants, "Get value at time", 1, t, 'Hertz', 'Linear')
        f2 = call(formants, "Get value at time", 2, t, 'Hertz', 'Linear')
        f3 = call(formants, "Get value at time", 3, t, 'Hertz', 'Linear')
        f4 = call(formants, "Get value at time", 4, t, 'Hertz', 'Linear')
        f1_list.append(f1)
        f2_list.append(f2)
        f3_list.append(f3)
        f4_list.append(f4)
    
    f1_list = [f1 for f1 in f1_list if str(f1) != 'nan']
    f2_list = [f2 for f2 in f2_list if str(f2) != 'nan']
    f3_list = [f3 for f3 in f3_list if str(f3) != 'nan']
    f4_list = [f4 for f4 in f4_list if str(f4) != 'nan']
    
    # calculate mean formants across pulses
    f1_mean = statistics.mean(f1_list)
    f2_mean = statistics.mean(f2_list)
    f3_mean = statistics.mean(f3_list)
    f4_mean = statistics.mean(f4_list)
    
    # calculate median formants across pulses, this is what is used in all subsequent calcualtions
    # you can use mean if you want, just edit the code in the boxes below to replace median with mean
    f1_median = statistics.median(f1_list)
    f2_median = statistics.median(f2_list)
    f3_median = statistics.median(f3_list)
    f4_median = statistics.median(f4_list)
    
    return f1_mean, f2_mean, f3_mean, f4_mean, f1_median, f2_median, f3_median, f4_median


## This function runs a 2-factor Principle Components Analysis (PCA) on Jitter and Shimmer

In [25]:
def runPCA(df):
    # z-score the Jitter and Shimmer measurements
    measures = ['localJitter', 'localabsoluteJitter', 'rapJitter', 'ppq5Jitter', 'ddpJitter',
                'localShimmer', 'localdbShimmer', 'apq3Shimmer', 'apq5Shimmer', 'apq11Shimmer', 'ddaShimmer']
    x = df.loc[:, measures].values
    x = StandardScaler().fit_transform(x)
    # PCA
    pca = PCA(n_components=2)
    principalComponents = pca.fit_transform(x)
    principalDf = pd.DataFrame(data = principalComponents, columns = ['JitterPCA', 'ShimmerPCA'])
    principalDf
    return principalDf

## This block of code runs the above functions on all of the '.wav' files in the /audio folder

In [26]:
# create lists to put the results
file_list = []
duration_list = []
mean_F0_list = []
sd_F0_list = []
hnr_list = []
localJitter_list = []
localabsoluteJitter_list = []
rapJitter_list = []
ppq5Jitter_list = []
ddpJitter_list = []
localShimmer_list = []
localdbShimmer_list = []
apq3Shimmer_list = []
aqpq5Shimmer_list = []
apq11Shimmer_list = []
ddaShimmer_list = []
f1_mean_list = []
f2_mean_list = []
f3_mean_list = []
f4_mean_list = []
f1_median_list = []
f2_median_list = []
f3_median_list = []
f4_median_list = []

# Go through all the wave files in the folder and measure all the acoustics
for wave_file in glob.glob(path_to_wav_file):
    print(wave_file)
    sound = parselmouth.Sound(wave_file)
    (duration, meanF0, stdevF0, hnr, localJitter, localabsoluteJitter, rapJitter, ppq5Jitter, ddpJitter, 
     localShimmer, localdbShimmer, apq3Shimmer, aqpq5Shimmer, apq11Shimmer, ddaShimmer) = measurePitch(
        sound, 75, 300, "Hertz")
    (f1_mean, f2_mean, f3_mean, f4_mean, f1_median, f2_median, f3_median, f4_median) = measureFormants(
        sound, wave_file, 75, 300)
    file_list.append(wave_file) # make an ID list
    duration_list.append(duration) # make duration list
    mean_F0_list.append(meanF0) # make a mean F0 list
    sd_F0_list.append(stdevF0) # make a sd F0 list
    hnr_list.append(hnr) #add HNR data
    
    # add raw jitter and shimmer measures
    localJitter_list.append(localJitter)
    localabsoluteJitter_list.append(localabsoluteJitter)
    rapJitter_list.append(rapJitter)
    ppq5Jitter_list.append(ppq5Jitter)
    ddpJitter_list.append(ddpJitter)
    localShimmer_list.append(localShimmer)
    localdbShimmer_list.append(localdbShimmer)
    apq3Shimmer_list.append(apq3Shimmer)
    aqpq5Shimmer_list.append(aqpq5Shimmer)
    apq11Shimmer_list.append(apq11Shimmer)
    ddaShimmer_list.append(ddaShimmer)
    
    # add the formant data
    f1_mean_list.append(f1_mean)
    f2_mean_list.append(f2_mean)
    f3_mean_list.append(f3_mean)
    f4_mean_list.append(f4_mean)
    f1_median_list.append(f1_median)
    f2_median_list.append(f2_median)
    f3_median_list.append(f3_median)
    f4_median_list.append(f4_median)

C:\Users\user\Desktop\MA Sprache und Kommunikation\2_Semester\Sprechstile\Angehelescu\Angehelescu_16_seg_00000000.wav
C:\Users\user\Desktop\MA Sprache und Kommunikation\2_Semester\Sprechstile\Angehelescu\Angehelescu_16_seg_00000001.wav
C:\Users\user\Desktop\MA Sprache und Kommunikation\2_Semester\Sprechstile\Angehelescu\Angehelescu_16_seg_00000002.wav
C:\Users\user\Desktop\MA Sprache und Kommunikation\2_Semester\Sprechstile\Angehelescu\Angehelescu_16_seg_00000003.wav
C:\Users\user\Desktop\MA Sprache und Kommunikation\2_Semester\Sprechstile\Angehelescu\Angehelescu_16_seg_00000004.wav
C:\Users\user\Desktop\MA Sprache und Kommunikation\2_Semester\Sprechstile\Angehelescu\Angehelescu_16_seg_00000005.wav
C:\Users\user\Desktop\MA Sprache und Kommunikation\2_Semester\Sprechstile\Angehelescu\Angehelescu_16_seg_00000006.wav
C:\Users\user\Desktop\MA Sprache und Kommunikation\2_Semester\Sprechstile\Angehelescu\Angehelescu_16_seg_00000007.wav
C:\Users\user\Desktop\MA Sprache und Kommunikation\2_Sem

## This block of code adds all of that data we just generated to a Pandas data frame

In [27]:
# Add the data to Pandas
df = pd.DataFrame(np.column_stack([file_list, duration_list, mean_F0_list, sd_F0_list, hnr_list, 
                                   localJitter_list, localabsoluteJitter_list, rapJitter_list, 
                                   ppq5Jitter_list, ddpJitter_list, localShimmer_list, 
                                   localdbShimmer_list, apq3Shimmer_list, aqpq5Shimmer_list, 
                                   apq11Shimmer_list, ddaShimmer_list, f1_mean_list, 
                                   f2_mean_list, f3_mean_list, f4_mean_list, 
                                   f1_median_list, f2_median_list, f3_median_list, 
                                   f4_median_list]),
                                   columns=['voiceID', 'duration', 'meanF0Hz', 'stdevF0Hz', 'HNR', 
                                            'localJitter', 'localabsoluteJitter', 'rapJitter', 
                                            'ppq5Jitter', 'ddpJitter', 'localShimmer', 
                                            'localdbShimmer', 'apq3Shimmer', 'apq5Shimmer', 
                                            'apq11Shimmer', 'ddaShimmer', 'f1_mean', 'f2_mean', 
                                            'f3_mean', 'f4_mean', 'f1_median', 
                                            'f2_median', 'f3_median', 'f4_median'])

#pcaData = runPCA(df) # Run jitter and shimmer PCA
#df = pd.concat([df, pcaData], axis=1) # Add PCA data
# reload the data so it's all numbers
df.to_csv("processed_results.csv", index=False)
df = pd.read_csv('processed_results.csv', header=0)
df.sort_values('voiceID').head(20)

,voiceID,duration,meanF0Hz,stdevF0Hz,HNR,localJitter,localabsoluteJitter,rapJitter,ppq5Jitter,ddpJitter,...,apq11Shimmer,ddaShimmer,f1_mean,f2_mean,f3_mean,f4_mean,f1_median,f2_median,f3_median,f4_median
0,C:\Users\user\Desktop\MA Sprache und Kommunika...,4.28,253.775735,21.862605,9.477288,0.020626,0.000081,0.011053,0.012469,0.033158,...,0.128099,0.161810,520.485980,1699.725220,2743.827297,3733.127595,504.845154,1669.064610,2695.226546,3890.653335
1,C:\Users\user\Desktop\MA Sprache und Kommunika...,5.96,230.974964,39.822558,9.240131,0.024346,0.000106,0.012157,0.014326,0.036472,...,0.128328,0.162375,577.390959,1712.439555,2702.162431,3781.595730,576.060829,1718.293899,2676.322306,3865.271638
2,C:\Users\user\Desktop\MA Sprache und Kommunika...,6.96,217.720590,40.656277,11.302664,0.017945,0.000082,0.008597,0.009552,0.025792,...,0.124668,0.146245,508.294443,1627.408990,2625.774276,3776.882800,490.967436,1598.774942,2578.613374,3863.210498
3,C:\Users\user\Desktop\MA Sprache und Kommunika...,6.96,223.013685,40.045658,10.033796,0.016122,0.000072,0.007982,0.009553,0.023945,...,0.140656,0.151124,606.841267,1742.728805,2690.430256,3776.089777,594.525926,1698.477353,2662.527361,3889.489510
4,C:\Users\user\Desktop\MA Sprache und Kommunika...,1.66,232.001187,25.032989,14.639509,0.014489,0.000063,0.007420,0.007130,0.022260,...,0.087254,0.111731,520.571708,1846.511330,2817.428073,3799.601199,510.810169,1698.532762,2725.957129,3878.346306
5,C:\Users\user\Desktop\MA Sprache und Kommunika...,1.32,250.428888,22.266818,11.042351,0.011820,0.000047,0.006609,0.007685,0.019826,...,0.111297,0.144012,647.544875,1797.001517,2676.586264,3767.479907,638.169583,1770.068888,2655.794075,3863.751058
6,C:\Users\user\Desktop\MA Sprache und Kommunika...,1.32,237.628499,36.104422,11.181819,0.020237,0.000086,0.009966,0.010336,0.029897,...,0.128031,0.191821,579.855282,1742.004822,2744.524214,3894.343897,537.892434,1659.205821,2776.132599,3932.810815
7,C:\Users\user\Desktop\MA Sprache und Kommunika...,1.50,248.235409,30.082759,10.303710,0.026496,0.000107,0.014192,0.014249,0.042575,...,0.149257,0.221331,520.453617,1961.139844,2738.263434,3786.310747,437.974302,2059.588295,2751.633696,3863.666536
8,C:\Users\user\Desktop\MA Sprache und Kommunika...,1.72,236.513569,32.893808,11.867408,0.016418,0.000070,0.007898,0.009991,0.023693,...,0.112141,0.134071,612.048393,1723.931546,2683.982263,3813.696917,618.882795,1712.016822,2714.095635,3890.949986
9,C:\Users\user\Desktop\MA Sprache und Kommunika...,1.34,240.318130,36.844530,13.396945,0.020450,0.000085,0.010956,0.012338,0.032868,...,0.106076,0.130140,614.850399,1377.197049,2686.264807,3718.394368,615.923471,1388.671854,2637.853254,3894.216809


## Next we calculate the vocal-tract length estimates

### Formant position
 Puts, D. A., Apicella, C. L., & Cárdenas, R. A. (2012). Masculine voices signal men's threat potential in forager and industrial societies. Proceedings of the Royal Society of London B: Biological Sciences, 279(1728), 601-609.

In [28]:
df['pF'] = (zscore(df.f1_median) + zscore(df.f2_median) + zscore(df.f3_median) + zscore(df.f4_median)) / 4

### Formant Dispersion
Fitch, W. T. (1997). Vocal tract length and formant frequency dispersion correlate with body size in rhesus macaques. The Journal of the Acoustical Society of America, 102(2), 1213-1222.

In [29]:
df['fdisp'] = (df['f4_median'] - df['f1_median']) / 3

### Fn (Average Formant)
Pisanski, K., & Rendall, D. (2011). The prioritization of voice fundamental frequency or formants in listeners’ assessments of speaker size, masculinity, and attractiveness. The Journal of the Acoustical Society of America, 129(4), 2201-2212.

In [30]:
df['avgFormant'] = (df['f1_median'] + df['f2_median'] + df['f3_median'] + df['f4_median']) / 4

### MFF 
Smith, D. R., & Patterson, R. D. (2005). The interaction of glottal-pulse rate and vocal-tract length in judgements of speaker size, sex, and age. The Journal of the Acoustical Society of America, 118(5), 3177-3186.

In [31]:
df['mff'] = (df['f1_median'] * df['f2_median'] * df['f3_median'] * df['f4_median']) ** 0.25

### Fitch VTL
Fitch, W. T. (1997). Vocal tract length and formant frequency dispersion correlate with body size in rhesus macaques. The Journal of the Acoustical Society of America, 102(2), 1213-1222.

In [32]:
# reload the data again
df.to_csv("processed_results.csv", index=False)
df = pd.read_csv('processed_results.csv', header=0)

df['fitch_vtl'] = ((1 * (35000 / (4 * df['f1_median']))) +
                   (3 * (35000 / (4 * df['f2_median']))) + 
                   (5 * (35000 / (4 * df['f3_median']))) + 
                   (7 * (35000 / (4 * df['f4_median'])))) / 4

### $\Delta$F 
Reby,D.,& McComb,K.(2003). Anatomical constraints generate honesty: acoustic cues to age and weight in the roars of red deer stags. Animal Behaviour, 65, 519e-530.

In [33]:
xysum = (0.5 * df['f1_median']) + (1.5 * df['f2_median']) + (2.5 * df['f3_median']) + (3.5 * df['f4_median'])
xsquaredsum = (0.5 ** 2) + (1.5 ** 2) + (2.5 ** 2) + (3.5 ** 2)
df['delta_f'] = xysum / xsquaredsum

### VTL($\Delta$F)
Reby,D.,&McComb,K.(2003).Anatomical constraints generate honesty: acoustic cues to age and weight in the roars of red deer stags. Animal Behaviour, 65, 519e-530.

In [34]:
df['vtl_delta_f'] = 35000 / (2 * df['delta_f'])

## Save the final data

In [35]:
# Write out the final dataframe
df.to_csv("processed_results2.csv", index=False)

## Run this to tell you when it's done

In [36]:
print("finished")

finished


In [37]:
df.to_pickle('../df_werte.pkl')